<a href="https://colab.research.google.com/github/florianraith/notebooks/blob/main/dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
src = "English"
tgt = "Dutch"

# See https://github.com/openlanguagedata/flores
src_filename = 'dev.eng_Latn'
tgt_filename = 'dev.nld_Latn'

In [61]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Flores/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Flores


In [62]:
import torch
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [77]:
src_data = pd.read_fwf(src_filename, header=None)
tgt_data = pd.read_fwf(tgt_filename, header=None)
print("Loaded files "  + src_filename + " and " + tgt_filename)

Loaded files dev.eng_Latn and dev.nld_Latn


In [64]:
src_data[:10]

,0
0,"On Monday, scientists from the Stanford Univer..."
1,Lead researchers say this may bring early dete...
2,The JAS 39C Gripen crashed onto a runway at ar...
3,The pilot was identified as Squadron Leader Di...
4,Local media reports an airport fire vehicle ro...
5,28-year-old Vidal had joined Barça three seaso...
6,"Since moving to the Catalan-capital, Vidal had..."
7,The protest started around 11:00 local time (U...
8,"Just after 11:00, protesters blocked traffic o..."
9,"At 11:20, the police asked the protesters to m..."


In [73]:
def sample_translations(n):
  # idx = torch.randint(0, len(src_data), (n, ))
  idx = torch.randperm(len(src_data))[:n]

  src_sample = src_data.loc[idx]
  tgt_sample = tgt_data.loc[idx]

  return list(zip(src_sample.iloc[:, 0].tolist(), tgt_sample.iloc[:, 0].tolist()))

sample_translations(1)

[('Photons are even smaller than the stuff that makes up atoms!',
  'फोटोनहरू परमाणु बनाउने सामानहरू भन्दा पनि झन साना हुन्छन्!')]

In [66]:
def generate_prompt(examples, few_shot_template, zero_shot_template, example_template, split):
  translations = sample_translations(examples + 1)
  examples_prompt = [
      example_template.replace("[src]", src)
      .replace("[tgt]", tgt)
      .replace("[x]", x)
      .replace("[y]", y)
      for (x, y) in translations[1:]
    ]

  prompt_template = few_shot_template if examples > 0 else zero_shot_template

  return (
    prompt_template
      .replace("[exm]", split.join(examples_prompt))
      .replace("[src]", src)
      .replace("[tgt]", tgt)
      .replace("[x]", translations[0][0])
  )

In [81]:
TEMPLATE_GPT = [
  "[exm]\n\n[src]: [x]\n[tgt]:",
  "[src]: [x]\n[tgt]:",
  "[src]: [x]\n[tgt]: [y]",
  "\n\n",
]

TEMPLATE_LLAMA = [
   "Given Examples: [exm], Translate '[x]' from [src] to [tgt]. Just output the translation. Do not output any additional information or notes. Output only the [tgt] sentence. Do not add any further examples.",
   "Translate '[x]' from [src] to [tgt]. Just output the translation. Do not output any additional information or notes. Output only the [tgt] sentence.",
   "[src]: '[x]' to [tgt]: '[y]'",
   ", ",
]

prompt = generate_prompt(2, *TEMPLATE_LLAMA)

prompt

'Given Examples: English: \'These children tend to get into a lot of trouble, because they "engage in risky behaviors, get into fights, and challenge authority" in order to stimulate their brain, since their brain can not be stimulated by normal methods.\' to Dutch: \'Deze kinderen werken zich vaak in de nesten omdat ze \'risicovol gedrag vertonen, ruzie maken en autoriteiten uitdagen\'. Dit doen ze om hun hersenen te stimuleren, omdat hun hersenen niet te stimuleren zijn met normale methoden.\', English: \'The world has over 5,000 different languages, including more than twenty with 50 million or more speakers.\' to Dutch: \'Er bestaan wereldwijd meer dan 5000 talen, waarvan meer dan twintig minimaal 50 miljoen sprekers hebben.\', Translate \'Only the effects of catastrophizing, not depression and fear was conditional of regular weekly structured PA sessions.\' from English to Dutch. Just output the translation. Do not output any additional information or notes. Output only the Dutch 